In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [9]:
df = pd.read_csv("Retail_Data_Transactions_.csv")

In [10]:
df.head()

,customer_id,trans_date,tran_amount
0,CS5295,2013-02-11,35
1,CS4768,2015-03-15,39
2,CS2122,2013-02-26,52
3,CS1217,2011-11-16,99
4,CS1850,2013-11-20,78


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125000 entries, 0 to 124999
Data columns (total 3 columns):
customer_id    125000 non-null object
trans_date     125000 non-null object
tran_amount    125000 non-null int64
dtypes: int64(1), object(2)
memory usage: 2.9+ MB


In [14]:
df['trans_date'] = pd.to_datetime(df['trans_date'],format='%Y-%m-%d')

In [16]:
df['trans_date'].min(), df['trans_date'].max()

(Timestamp('2011-05-16 00:00:00'), Timestamp('2015-03-16 00:00:00'))

In [18]:
# Compute the maximum date to know the latest transaction date

max_date = max(df['trans_date'])
max_date

Timestamp('2015-03-16 00:00:00')

In [22]:
df['Difference'] = max_date - df['trans_date']

df['Difference'] = df['Difference'].dt.days

In [23]:
df.head()

,customer_id,trans_date,tran_amount,Difference
0,CS5295,2013-02-11,35,763
1,CS4768,2015-03-15,39,1
2,CS2122,2013-02-26,52,748
3,CS1217,2011-11-16,99,1216
4,CS1850,2013-11-20,78,481


In [24]:
df_last2yrs=df[df['Difference'] < 730]
df_last2yrs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65023 entries, 1 to 124999
Data columns (total 4 columns):
customer_id    65023 non-null object
trans_date     65023 non-null datetime64[ns]
tran_amount    65023 non-null int64
Difference     65023 non-null int64
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 2.5+ MB


In [25]:
rfm_test = df.groupby('customer_id').agg({'Difference': lambda x: x.min(), # Recency
                                        'customer_id': lambda x: len(x),# Frequency
                                        'tran_amount': lambda x: x.sum()})  # Monetary Value

rfm_test.rename(columns={'Difference': 'Recency', 
                         'customer_id': 'Frequency', 
                         'tran_amount': 'Monetary_value'}, inplace=True)

In [28]:
rfm_test.head()

,Recency,Frequency,Monetary_value
customer_id,,,
CS1112,61,15,1012
CS1113,35,20,1490
CS1114,32,19,1432
CS1115,11,22,1659
CS1116,203,13,857


In [30]:
rfm_test.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,6889.0,80.538249,85.382526,0.0,22.0,53.0,111.0,857.0
Frequency,6889.0,18.144869,5.193014,4.0,14.0,18.0,22.0,39.0
Monetary_value,6889.0,1179.269705,465.832609,149.0,781.0,1227.0,1520.0,2933.0


In [31]:
quartiles = rfm_test.quantile(q=[0.25,0.50,0.75])
print(quartiles, type(quartiles))

      Recency  Frequency  Monetary_value
0.25     22.0       14.0           781.0
0.50     53.0       18.0          1227.0
0.75    111.0       22.0          1520.0 <class 'pandas.core.frame.DataFrame'>


In [32]:
quartiles=quartiles.to_dict()
quartiles

{'Recency': {0.25: 22.0, 0.5: 53.0, 0.75: 111.0},
 'Frequency': {0.25: 14.0, 0.5: 18.0, 0.75: 22.0},
 'Monetary_value': {0.25: 781.0, 0.5: 1227.0, 0.75: 1520.0}}

In [39]:
## for Recency 

def R_class(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
## for Frequency and Monetary value 

def FM_class(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1    

In [40]:
rfmSeg = rfm_test
rfmSeg['R_Quartile'] = rfmSeg['Recency'].apply(R_class, args=('Recency',quartiles,))
rfmSeg['F_Quartile'] = rfmSeg['Frequency'].apply(FM_class, args=('Frequency',quartiles,))
rfmSeg['M_Quartile'] = rfmSeg['Monetary_value'].apply(FM_class, args=('Monetary_value',quartiles,))

In [41]:
rfmSeg.head()

,Recency,Frequency,Monetary_value,R_Quartile,F_Quartile,M_Quartile
customer_id,,,,,,
CS1112,61,15,1012,3,3,3
CS1113,35,20,1490,2,2,2
CS1114,32,19,1432,2,2,2
CS1115,11,22,1659,1,2,1
CS1116,203,13,857,4,4,3


In [45]:
rfmSeg.to_excel("rfm_test_data1.xlsx")

In [44]:
rfmSeg[rfmSeg['R_Quartile'] == 4].shape

(1711, 6)